In [2]:
import cv2
import torch
from RealESRGAN import RealESRGAN
import numpy as np
import os

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = RealESRGAN(device, scale=2)
model.load_weights('weights/RealESRGAN_x4plus.pth', download=True) 

### Video Upscaling using Real-ESRGAN (image-to-image) technique 

In [4]:
def process_video(input_file_path, output_file_path):
    cap = cv2.VideoCapture(input_file_path) #file is opened and read
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # width and height of video is obtained
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    out_width = width * 2  # upscale to double
    out_height = height * 2
    fourcc = cv2.VideoWriter_fourcc(*"HVEC") # codec selection based on your wish can use mp4v,etc
    out = cv2.VideoWriter(output_file_path, fourcc, cap.get(cv2.CAP_PROP_FPS), (out_width, out_height)) # final video format name,dimensions and fps

    frame_counter = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # convert frame to rgb
        sr_frame_tensor = model.predict(frame_rgb) # enhance using Real-ESRGAN model 
        sr_frame = np.array(sr_frame_tensor) # convert frame to np array
        sr_frame = cv2.cvtColor(sr_frame, cv2.COLOR_RGB2BGR) # converting to blue,green,red format
        sr_frame = cv2.cvtColor(sr_frame, cv2.COLOR_BGR2HSV) #converting to Hue, Saturation, Value for color correction
        sr_frame = np.clip(sr_frame * np.array([1, 1.1, 1.05]), 0, 255).astype(np.uint8) # here the contrast and brightness is adjusted to 8bit for better output
        sr_frame = cv2.cvtColor(sr_frame, cv2.COLOR_HSV2BGR) # again converted from hsv to rgb format

        frame_counter += 1
        print(f"Frame {frame_counter} upscaled successfully!")

        out.write(sr_frame) # the frame is witten to outfile

    cap.release()
    out.release()
    cv2.destroyAllWindows()

### Video Resize for desired dimension

In [5]:
def resize_video(input_path, output_path, new_resolution):
    cap = cv2.VideoCapture(input_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*"HVEC")
    out = cv2.VideoWriter(output_path, fourcc, fps, new_resolution)

    while True:
        ret, frame = cap.read()
        if not ret: 
            break

        resized_frame = cv2.resize(frame, new_resolution)
        out.write(resized_frame)

    cap.release()
    out.release()
    cv2.destroyAllWindows()

### MAIN BLOCK

In [ ]:
input_video_path = "./1.mp4"  # change the input file location to upscale
output_video_path = "./output1.mp4" # change to your desired name as its temp
final_output_path="./output_final1.mp4" # this is final output file name
new_resolution = (1280, 720) # this is the desired resize video dimensions
process_video(input_video_path, output_video_path)
resize_video(output_video_path, final_output_path, new_resolution)
os.remove(output_video_path)

### FAQ
#### Why Real-ESRGAN?
##### Real-ESRGAN is an image-to-image technique well-suited for our frame-by-frame video upscaling approach. Its use of Generative Adversarial Networks (GANs) allows it to handle content filling by generating plausible textures and details, seamlessly extending the missing left and right sides of the video frames.